<a href="https://colab.research.google.com/github/ronnieogwang/Self_Driving_car_course/blob/master/section10_behavioralcloning/drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from flask import Flask
import socketio
import eventlet
from keras.models import load_model
sio = socketio.Server()
import numpy as np
import base64
from io import BytesIO
from PIL import Image
import cv2


In [0]:
app = Flask(__name__)

speed_limit = 10

def img_preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [0]:
#collect real time data from simulator
@sio.on('telemetry')
def telemetry(sid, data):
	speed = float(data['speed'])
	image = Image.open(BytesIO(base64.b64decode(data['image'])))
	image = np.asarray(image)  #convert image to array
	image = img_preprocess(image)
	image = np.array([image])  #convert image to 4D
	steering_angle = float(model.predict(image))
	throttle = 1.0 - speed/speed_limit
	print('{} {} {}'.format(steering_angle, throttle, speed))
	send_control(steering_angle, throttle)



In [0]:
#connect to simulator
@sio.on('connect')
def connect(sid, myenviron):
    print('connected')
    send_control(2,0)


In [0]:
#send controls to the car
def send_control(steering_angle, throttle):
    sio.emit('steer', data = {
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__()
        })
    

In [0]:
if __name__ == '__main__':
    model = load_model('model.h5')
    app = socketio.Middleware(sio, app)
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)
